In [34]:
%pip install torch transformers[torch] datasets accelerate evaluate bitsandbytes scikit-learn huggingface_hub trl peft ipywidgets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.kill(os.getpid(), 9)

: 

: 

In [1]:
import os
from pathlib import Path
import csv

import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn.functional as F

from huggingface_hub import notebook_login
from datasets import Dataset, DatasetDict
import evaluate

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

2025-03-05 11:29:36.662638: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_folder = '/kaggle/input/clef2025-checkthat/data' # data # /workspace/data
dataset = pd.DataFrame(columns=['sentence_id','sentence','label','lang','split'])

for language in os.listdir(data_folder):
    for filename in os.listdir(f"{data_folder}{os.sep}{language}"):
        if '.tsv' in filename:
            abs_path = f"{data_folder}{os.sep}{language}{os.sep}{filename}"
            df = pd.read_csv(abs_path, sep='\t', quoting=csv.QUOTE_NONE)
            if 'solved_conflict' in df.columns:
                df.drop(columns=['solved_conflict'], inplace=True)
            df['lang'] = language
            df['split'] = Path(filename).stem
            dataset = pd.concat([dataset, df], axis=0)

In [3]:
dataset = dataset[dataset['lang'] == 'english']

In [4]:
train = dataset[dataset['split'].str.contains('train')].copy()
dev = dataset[dataset['split'].str.contains('dev')].copy()
test = dataset[dataset['split'].str.contains('dev_test')].copy()

print(f"Train: {train.shape}")
print(f"Dev: {dev.shape}")
print(f"Test: {test.shape}")


Train: (830, 5)
Dev: (946, 5)
Test: (484, 5)


In [5]:
print(f"Train: {train['label'].value_counts(normalize=True)}")
print(f"Dev: {dev['label'].value_counts(normalize=True)}")
print(f"Test: {test['label'].value_counts(normalize=True)}")

Train: label
OBJ     0.640964
SUBJ    0.359036
Name: proportion, dtype: float64
Dev: label
OBJ     0.617336
SUBJ    0.382664
Name: proportion, dtype: float64
Test: label
OBJ     0.747934
SUBJ    0.252066
Name: proportion, dtype: float64


In [6]:
train.loc[:, 'label'] = train['label'].apply(lambda x: 0 if x == 'OBJ' else 1)
dev.loc[:, 'label'] = dev['label'].apply(lambda x: 0 if x == 'OBJ' else 1)
test.loc[:, 'label'] = test['label'].apply(lambda x: 0 if x == 'OBJ' else 1)

In [7]:
train['label'] = train['label'].astype(int)
dev['label'] = dev['label'].astype(int)
test['label'] = test['label'].astype(int)

In [8]:
train.head()

,sentence_id,sentence,label,lang,split
0,b9e1635a-72aa-467f-86d6-f56ef09f62c3,Gone are the days when they led the world in r...,1,english,train_en
1,f99b5143-70d2-494a-a2f5-c68f10d09d0a,The trend is expected to reverse as soon as ne...,0,english,train_en
2,4076639c-aa56-4202-ae0f-9d9217f8da68,But there is the specious point again.,0,english,train_en
3,b057c366-698e-419d-a284-9b16d835c64e,He added he wouldn’t be surprised to see a new...,0,english,train_en
4,a5a9645e-7850-41ba-90a2-5def725cd5b8,"Not less government, you see; the same amount ...",1,english,train_en


In [9]:
notebook_login()

In [9]:
# Converting pandas DataFrames into Hugging Face Dataset objects:
df_train = Dataset.from_pandas(train.drop(['sentence_id', 'lang', 'split'],axis=1).reset_index(drop=True))
df_dev = Dataset.from_pandas(dev.drop(['sentence_id', 'lang', 'split'],axis=1).reset_index(drop=True))
df_test = Dataset.from_pandas(test.drop(['sentence_id', 'lang', 'split'],axis=1).reset_index(drop=True))


In [10]:
dataset = DatasetDict({
    'train': df_train,
    'dev': df_dev,
    'test': df_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 830
    })
    dev: Dataset({
        features: ['sentence', 'label'],
        num_rows: 946
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 484
    })
})

In [11]:
df_train[0]

{'sentence': 'Gone are the days when they led the world in recession-busting',
 'label': 1}

In [12]:
train.label.value_counts(normalize=True)

label
0    0.640964
1    0.359036
Name: proportion, dtype: float64

In [13]:
class_weights = compute_class_weight('balanced', classes=np.unique(dataset['train']['label']), y=dataset['train']['label'])
class_weights

array([0.78007519, 1.39261745])

In [14]:
model_name = "meta-llama/Llama-3.2-1B"

In [15]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = False, # enable 4-bit quantization
    #bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    #bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

#lora_config = LoraConfig(
#    r = 16, # the dimension of the low-rank matrices
#    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
#    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
#    lora_dropout = 0.05, # dropout probability of the LoRA layers
#    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
#    task_type = 'SEQ_CLS'
#)

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    #quantization_config=quantization_config,
    num_labels=2,
    id2label={0: 'OBJ', 1: 'SUBJ'}, 
    label2id={'OBJ': 0, 'SUBJ': 1},
    output_attentions = False,
    output_hidden_states = False
)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
   

In [18]:
#model = prepare_model_for_kbit_training(model)
#model

In [19]:
#model = get_peft_model(model, lora_config)
#model

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [21]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [22]:
MAX_LEN = 256

def llama_preprocessing_function(texts):
    return tokenizer(texts['sentence'], padding=True, truncation=True, max_length=MAX_LEN, return_tensors='pt')

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True)

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/946 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

In [23]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
def evaluate_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    m_prec, m_rec, m_f1, m_s = precision_recall_fscore_support(labels, predictions, average="macro",
                                                                zero_division=0)
    p_prec, p_rec, p_f1, p_s = precision_recall_fscore_support(labels, predictions, labels=[1],
                                                                zero_division=0)

    return {
        'macro_F1': m_f1,
        'macro_P': m_prec,
        'macro_R': m_rec,
        'SUBJ_F1': p_f1[0],
        'SUBJ_P': p_prec[0],
        'SUBJ_R': p_rec[0],
        'accuracy': acc
    }

In [25]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [26]:
# Define training args
training_args = TrainingArguments(
    output_dir=f"/workspace/models",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    num_train_epochs=5,
    #weight_decay=0.1,
    eval_strategy="epoch",
    save_strategy="no",
    save_total_limit=1,
    #warmup_ratio=0.5,
    #load_best_model_at_end=True,
    metric_for_best_model="macro_F1",
)

In [27]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['dev'],
    data_collator = collate_fn,
    compute_metrics = evaluate_metrics,
    class_weights=class_weights,
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [33]:
predictions, labels, _ = trainer.predict(tokenized_datasets['test'])
evaluate_metrics((predictions, labels))

{'macro_F1': 0.7227893562972096,
 'macro_P': 0.8009254408940107,
 'macro_R': 0.6951136672402862,
 'SUBJ_F1': 0.5549738219895288,
 'SUBJ_P': 0.7681159420289855,
 'SUBJ_R': 0.4344262295081967,
 'accuracy': 0.8243801652892562}